In [46]:
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [47]:
import requests
import json
import pandas as pd
import datetime
#spotify API works with 64base urls
import base64
import time

# spotify requests function
setting the relevant data that needed to get an access token for spotify API requests 

In [105]:
def spotify_req():
    client_id = "0cd5c209ad8d46b291518e5f5243ba22"
    client_secret = "91b4c2782052423493edadd09067d4a1"
    client_creds=f"{client_id}:{client_secret}"
    client_creds_b64=base64.b64encode(client_creds.encode())
    token_url="https://accounts.spotify.com/api/token"
    method = "POST"
    token_data = {
        "grant_type":"client_credentials"   
    }
    token_headers={
        "Authorization":f"Basic {client_creds_b64.decode()}"# <base64 encoded client_id:client_secret>
    }
    r=requests.post(token_url,data=token_data,headers=token_headers)
    print(r.json())
    valid_request = r.status_code in range(200,299) #return's boolean 
    # Set the authorization header with the access token
    headers = {
        "Authorization": f"Bearer {r.json()['access_token']}"
    }
    return headers
headers = spotify_req()

{'access_token': 'BQCxvXYT4Isi87xWHTfoc2Iz_oSQu4mzEdEEOVf-R9srccVSxLtkWpzfrAS_AG6zaM_Qn43t2SDpKoiAXjsopKsuZHb-_u-1ZcfN9wV-ebMAJWY4Dyo', 'token_type': 'Bearer', 'expires_in': 3600}


# generate list of songs from spotify
setting the base url of spotify and the endpoint to the search API

In [49]:
# Set up the API base URL and the API endpoint for searching for songs
api_base_url = "https://api.spotify.com"
search_endpoint = "/v1/search"

making a search API request, getting a list of songs according to the years of billboard charts we crawled befor,
then saving it to a dataframe

In [24]:
# generate a flops song from spotify api, songs that are relevant for the years of the list we are working on
artist = []
song_name = []
spotify_track_id = []
start_index = 0
for i in range (20):
    time.sleep(4.5)
    params = {
    "q": 'year:2013-2018',
    "type":"track",
    "maket": "US",
    "limit": 50,
    "offset": start_index
    }
    response = requests.get(api_base_url + search_endpoint, headers=headers, params=params)
    if response.status_code not in range(200,299):
        print(response.headers)
        print(response.headers.get('Retry-After'))
        break
    else:
    # add the data returned by the API to ids array
        for j in range(50):
            data = json.loads(response.text)
            artist.append(data["tracks"]["items"][j]["artists"][0]["name"])
            song_name.append(data["tracks"]["items"][j]["name"])
            spotify_track_id.append(data["tracks"]["items"][j]["id"])
    start_index = start_index + 50
start_index = 0  
for i in range (20):
    time.sleep(4.5)
    params = {
    "q": 'year:2019-2022',
    "type":"track",
    "maket": "US",
    "limit": 50,
    "offset": start_index
    }
    response = requests.get(api_base_url + search_endpoint, headers=headers, params=params)
    if response.status_code != 200:
        print(response.headers)
        print(response.headers.get('Retry-After'))
        break
    else:
    # add the data returned by the API to ids array
        for j in range(50):
            data = json.loads(response.text)
            artist.append(data["tracks"]["items"][j]["artists"][0]["name"])
            song_name.append(data["tracks"]["items"][j]["name"])
            spotify_track_id.append(data["tracks"]["items"][j]["id"])
    start_index = start_index + 50
df_flops = pd.DataFrame({"artist": artist, "song_name": song_name , "spotify_track_id": spotify_track_id})
df_flops

,artist,song_name,spotify_track_id
0,The Neighbourhood,Sweater Weather,2QjOHCTQ1Jl3zawyYOpxh6
1,Ariana Grande,Santa Tell Me,0lizgQ7Qw35od7CYaoMBZb
2,J. Cole,No Role Modelz,68Dni7IE4VyPkTOH9mRWHr
3,Steve Lacy,Dark Red,3EaJDYHA0KnX88JvDhL9oa
4,Kelly Clarkson,Underneath the Tree,3YZE5qDV7u1ZD1gZc47ZeR
...,...,...,...
1995,Burna Boy,Alone,0AoBY2Y3qs6dtGgOD6c91N
1996,Mother Mother,Hayloft II,0kYMwaQWABTkFff8AZjmYI
1997,The Backseat Lovers,Pool House,74f0dpqHhTeTJzs4pmZ1yq
1998,Kelly Clarkson,Under The Mistletoe,2K4XbdyLxi9jhqVQeyUKRK


Remove duplicates songs that we got from the API request

In [25]:
len(df_flops['spotify_track_id'].unique())

1946

In [27]:
# remove duplicates songs from df
df_flops.drop_duplicates(subset = ['spotify_track_id'], inplace = True)
df_flops.shape

(1946, 3)

Removing songs from flops that are hits

In [31]:
#If a song, that was suggest by spotify, and added to the flops df, has a spotfy songID that is in our Billboard df,
billboard_df = pd.read_csv("../Data/03 billboard_scraped_w featuers_popular1.csv")
billboard_songs_ids = billboard_df['spotify_track_id'].tolist()
flops_ids = df_flops['spotify_track_id'].tolist()
no_dups = [id for id in flops_ids if id not in billboard_songs_ids]
len(no_dups)

1482

In [43]:
# REMOVE FROM FLOPS the "hits" from flops df
flops_final_data = df_flops[(df_flops["spotify_track_id"]).isin(no_dups)]
flops_final_data.reset_index(drop=True, inplace = True)

In [109]:
# # Save new FLOPS DataFrame
# flops_final_data.to_csv("flops_data.csv")

In [66]:
All_flops = pd.read_csv("flops_data.csv")
flops_df_pt_01 = All_flops.iloc[:921]
flops_df_pt_02 = All_flops.iloc[921:]
flops_df_pt_02.shape
flops_df_pt_02.head(10)

(561, 4)

,Unnamed: 0,artist,song_name,spotify_track_id
921,921,Naturelle,Falling Asleep,5N8KsYJ8IEMQVw33JvytW8
922,922,PinkPantheress,Pain,4S4QJfBGGrC8jRIjJHf1Ka
923,923,Bad Omens,Just Pretend,1H4Y9uW4N0LsxJUz0VnaPJ
924,924,Bailey Zimmerman,Where It Ends,7lNezbnIXBbgNmFOa9dt1c
925,925,Kendrick Lamar,Rich Spirit,1QPreu0BNOrUfEb8HTd2qG
926,926,Mareux,The Perfect Girl,5RBOcBpJXaNnHCGViJmYhh
927,927,The Weeknd,Out of Time,2SLwbpExuoBDZBpjfefCtV
928,928,Kendrick Lamar,Die Hard,2g6tReTlM2Akp41g0HaeXN
929,929,Lainey Wilson,Heart Like A Truck,5ZCjp56T8J6d7amJyFSzrI
930,930,J. Cole,m y . l i f e (with 21 Savage & Morray),1D3z6HTiQsNmZxjl7F7eoG


# add audio features
setting the endpoint to be the get tracks audio features endpoint, making lists for every feature and cheaking for missing data that the API request did not return 

In [71]:
# set the audio features end point
audio_features_endpoint = "/v1/audio-features"

#set the arrays for the audio features data response
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []
time_signature = []
missing_data_ids=[]

#set loop for extracting the audio featureas data for each track in df
for spot_id in flops_df_pt_02['spotify_track_id']:
    response = requests.get(api_base_url + audio_features_endpoint + '/' + f"{spot_id}", headers=headers)
    if response.status_code == 404:
            print(spot_id)
            missing_data_ids.append(spot_id)
            continue
    if response.status_code not in range(200,299):
        print(response.headers)
        print(response.headers.get('Retry-After'))

        break
    else:
        data = json.loads(response.text)
        danceability.append(data["danceability"])
        energy.append(data["energy"])
        key.append(data["key"])
        loudness.append(data["loudness"])
        mode.append(data["mode"])
        speechiness.append(data["speechiness"])
        acousticness.append(data["acousticness"])
        instrumentalness.append(data["instrumentalness"])
        liveness.append(data["liveness"])
        valence.append(data["valence"])
        tempo.append(data["tempo"])
        duration_ms.append(data["duration_ms"])
        time_signature.append(data["time_signature"])
        time.sleep(4.5)

{'www-authenticate': 'Bearer realm="spotify", error="invalid_token", error_description="The access token expired"', 'access-control-allow-origin': '*', 'access-control-allow-headers': 'Accept, App-Platform, Authorization, Content-Type, Origin, Retry-After, Spotify-App-Version, X-Cloud-Trace-Context, client-token, content-access-token', 'access-control-allow-methods': 'GET, POST, OPTIONS, PUT, DELETE, PATCH', 'access-control-allow-credentials': 'true', 'access-control-max-age': '604800', 'content-type': 'application/json', 'content-encoding': 'gzip', 'strict-transport-security': 'max-age=31536000', 'x-content-type-options': 'nosniff', 'date': 'Fri, 13 Jan 2023 22:08:00 GMT', 'server': 'envoy', 'Via': 'HTTP/2 edgeproxy, 1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'Transfer-Encoding': 'chunked'}
None
{'access_token': 'BQATW1yVNnE1CKa8-oOgTRfnAhDk3l1zwLcdv8ZxWcoEC0By76Cz0WDqdFIERsnbaVvXEkEXDYU2QstVXcMPeq-BWWiBcKt5UBCo8mKEvEx5dSppF_Q', 'token_type': 'Bearer', '

In [70]:
for d in missing_data_ids:
    print(d)
    flops_df_pt_02 = flops_df_pt_02[flops_df_pt_02['spotify_track_id'].str.contains(d) == False]
    if d not in flops_df_pt_02['spotify_track_id']:
                print(f"{d} is not in DataFrame")
print("Done")

6IEpMvQOYDtEyVE5ZOP9UA
6IEpMvQOYDtEyVE5ZOP9UA is not in DataFrame
2ex3O9bv4muJcuVI3BQbtj
2ex3O9bv4muJcuVI3BQbtj is not in DataFrame
Done


adding the new data to our dataframe 

In [72]:
#add featurs to DataFrame
flops_with_fearures_pt2 = flops_df_pt_02.assign(**{"danceability": danceability,"energy": energy, "key": key, "loudness": loudness,
             "mode": mode, "speechiness": speechiness, "acousticness": acousticness, "instrumentalness":instrumentalness,
            "liveness": liveness, "valence": valence, "tempo": tempo, "duration_ms": duration_ms,
             "time_signature": time_signature})

# add popularity feature
setting the endpoint to be get track endpoint, sending a list of 50 songs in each request then addind the data to the dataframe

In [106]:
#request to API, for the: popularity property
search_endpoint = "/v1/tracks/"
i=0
j=50
track_ids = []
tracks_popularity = []
for page in range(30):
    time.sleep(4.5)
    track_ids = all_flops_with_fearures.iloc[i:j]["spotify_track_id"].tolist()
    params = {
        "ids": ','.join(track_ids),
        "market": "US"
    }
      # Send the request to the API
    response = requests.get(api_base_url + search_endpoint, headers=headers, params=params)
    if response.status_code not in range(200,299):
        print(response.text)
        headers = spotify_req()
        response = requests.get(api_base_url + tracks_endpoint + f"{spot_id}", headers=headers)
        data = response.json()
    else:
        # add the data returned by the API to ids array
        data = json.loads(response.text)
        if page == 29:
            for f in range (29):
                tracks_popularity.append(data["tracks"][f]["popularity"])
        else:
            for f in range (50):
                tracks_popularity.append(data["tracks"][f]["popularity"])
    i=i+50
    j=i+50

In [107]:
all_flops_with_fearures = all_flops_with_fearures.assign(**{"tracks_popularity": tracks_popularity})
all_flops_with_fearures

,artist,song_name,spotify_track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,release_year,tracks_popularity
0,Ariana Grande,Santa Tell Me,0lizgQ7Qw35od7CYaoMBZb,0.525,0.621,7,-7.364,1,0.1160,0.04890,0.000000,0.2940,0.591,191.900,204093,4,2014,93
1,J. Cole,No Role Modelz,68Dni7IE4VyPkTOH9mRWHr,0.690,0.521,10,-8.492,0,0.3390,0.32400,0.000000,0.0534,0.494,100.028,292799,4,2014,90
2,Steve Lacy,Dark Red,3EaJDYHA0KnX88JvDhL9oa,0.603,0.784,6,-4.023,1,0.0620,0.44600,0.000008,0.1190,0.769,172.041,173104,4,2017,90
3,Kelly Clarkson,Underneath the Tree,3YZE5qDV7u1ZD1gZc47ZeR,0.509,0.812,8,-5.439,1,0.0537,0.00128,0.000020,0.2140,0.689,159.878,229640,4,2013,82
4,The Weeknd,Die For You,2Ch7LmS7r2Gy2kc64wv3Bz,0.582,0.525,1,-7.163,0,0.0627,0.11100,0.000000,0.1340,0.510,133.249,260253,4,2016,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,Burna Boy,Alone,0AoBY2Y3qs6dtGgOD6c91N,0.600,0.659,4,-7.264,0,0.0542,0.17600,0.000000,0.1110,0.307,89.955,221747,4,2022,83
1478,Mother Mother,Hayloft II,0kYMwaQWABTkFff8AZjmYI,0.326,0.866,9,-4.541,0,0.1010,0.00440,0.000007,0.0912,0.172,178.181,215040,4,2022,78
1479,The Backseat Lovers,Pool House,74f0dpqHhTeTJzs4pmZ1yq,0.594,0.355,7,-11.184,1,0.0305,0.04780,0.000109,0.0559,0.327,134.097,269412,4,2019,72
1480,Kelly Clarkson,Under The Mistletoe,2K4XbdyLxi9jhqVQeyUKRK,0.437,0.802,2,-5.063,1,0.1110,0.01680,0.000000,0.0860,0.704,178.137,225688,4,2020,75


In [108]:
# all_flops_with_fearures.to_csv("../Data/04 all_flops_w featuers_popular.csv")